In [ ]:
# IJulia.qtconsole();

In [1]:
import Pkg
Pkg.activate(".")
# Pkg.instantiate()

"C:\\Users\\Robert\\Promotion\\Workshop\\02_cc_dlmp_varaware\\01_case_study_cc_dlmp\\Project.toml"

In [2]:
using Logging
using DataFrames, CSV, LinearAlgebra
using JuMP
# using Gurobi, Ipopt
using Mosek, MosekTools

In [17]:
include("src/input.jl");
include("src/model_definition.jl")

build_model (generic function with 2 methods)

In [4]:
datadir = ("data/feeder_data/basecase_noneg/")
# datadir = ("data/feeder_data/feeder141/")
# datadir = ("data/feeder_data/feeder141_lv/")

feeder = load_feeder(datadir);

>>>>> Reading feeder data from data/feeder_data/basecase_noneg/


In [6]:
relative_std = 0.1
loads = [b.d_P for b in feeder.buses]
std_vec = loads .* relative_std
var_vec = std_vec.^2
Σ = diagm(0 => var_vec)

casesettings = Dict(
    "var_vec" => var_vec,
    "Σ" => Σ,
    "z_g" => 1.945 ,
    "z_v" => 1.945,
    "toggle_volt_cc" => true,
    "toggle_gen_cc" => true,
    "vfac" => 0,
    "qcfac" => 1,
    "output_level" => 0,
    "Ψ" => 0,
)

Dict{String,Any} with 10 entries:
  "var_vec"        => [0.0, 0.00629801, 0.0, 4.0401e-6, 2.9929e-6, 8.4681e-6, 4…
  "toggle_volt_cc" => true
  "qcfac"          => 1
  "Ψ"              => 0
  "Σ"              => [0.0 0.0 … 0.0 0.0; 0.0 0.00629801 … 0.0 0.0; … ; 0.0 0.0…
  "toggle_gen_cc"  => true
  "output_level"   => 0
  "z_g"            => 1.945
  "z_v"            => 1.945
  "vfac"           => 0

In [18]:
m = build_model(feeder, casesettings)

A JuMP Model
Minimization problem with:
Variables: 121
Objective function type: GenericAffExpr{Float64,VariableRef}
`VariableRef`-in-`MathOptInterface.GreaterThan{Float64}`: 47 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.EqualTo{Float64}`: 99 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.GreaterThan{Float64}`: 5 constraints
`GenericAffExpr{Float64,VariableRef}`-in-`MathOptInterface.LessThan{Float64}`: 33 constraints
`Array{GenericAffExpr{Float64,VariableRef},1}`-in-`MathOptInterface.SecondOrderCone`: 15 constraints
Model mode: AUTOMATIC
CachingOptimizer state: EMPTY_OPTIMIZER
Solver name: Mosek
Names registered in the model: fp, fq, gp, gp0_minus, gp0_plus, gq, linear_cost, r, t, v, variance_penalty, α, β, γ, δm, δp, ζ, η, θm, θp, λ, μm, μp, π, ρ

In [19]:
optimize!(m)
status = termination_status(m)

MOSEK warning 705: #3 (nearly) zero elements are specified in sparse row ''(87) of matrix 'A'.
MOSEK warning 705: #4 (nearly) zero elements are specified in sparse row ''(93) of matrix 'A'.


OPTIMAL::TerminationStatusCode = 1

In [ ]:
# Capture Results
results_df = DataFrame()

b_idx = []
gp_res = []
gq_res = []
voltages = []
alphas = []
lambdas = []
pies = []
delta_plus, delta_minus = [], []
mu_plus, mu_minus = [], []
gamma = []
etas = []
voltvar = []

num_approx = 1e-7

for b in bus_set
    push!(b_idx, b) 
    push!(gp_res, (abs(value(gp[b])) - num_approx) > 0 ? value(gp[b]) : 0 )
    push!(gq_res, (abs(value(gq[b])) - num_approx) > 0 ? value(gq[b]) : 0 )
    push!(voltages, sqrt(value(v[b])))
    if any_cc 
        push!(alphas, (abs(value(α[b])) - num_approx) > 0 ? value(α[b]) : 0 ) 
    else
        push!(alphas, 0)
    end
    push!(lambdas, shadow_price(λ[b]))
    push!(pies, shadow_price(π[b]))
    if b in setdiff(gen_buses,[root_bus])
        push!(delta_plus, shadow_price(δp[b]))
        push!(delta_minus, shadow_price(δm[b]))
    else
        push!(delta_plus, 0)
        push!(delta_minus, 0)
    end
    
    if b == root_bus
        push!(mu_plus, 0)
        push!(mu_minus, 0)
    else
        push!(mu_plus, shadow_price(μp[b]))
        push!(mu_minus, shadow_price(μm[b]))
    end
    if !any_cc || b == root_bus
        push!(voltvar, 0)
        push!(etas, 0)
    else
        push!(voltvar, value(t[bus_to_idx[b]]))
        push!(etas, shadow_price(η[bus_to_idx[b]]))
    end
end
objective = zeros(n_buses)
objective[root_bus] = objective_value(m)
gamma = zeros(n_buses)
if any_cc
    gamma[root_bus] = shadow_price(γ)
end

results_df = DataFrame(
    objective = objective,
    bus = b_idx, 
    gp = gp_res,
    gq = gq_res,
    voltage = voltages,
    alpha = alphas,
    lambda = lambdas,
    pi = pies,
    gamma = gamma,
    eta = etas,
    delta_plus = delta_plus,
    delta_minus = delta_minus,
    mu_plus = mu_plus,
    mu_minus = mu_minus,
    voltvar = voltvar,
)


In [ ]:
saveres = results_df

In [ ]:
CSV.write("figures/141res_lv.csv", saveres)

In [ ]:
mu = results_df[:mu_plus] .+ results_df[:mu_minus]
mu = mu[2:end]

b = 1 ./ diag(C)
b = [(x == Inf ? 0 : x) for x in b]

g3 = 2*s*z*e'*R'*mu/sum(b)

g1 = s^2/sum(b) 

delta = results_df[:delta_plus] .+ results_df[:delta_minus]
detlta = delta[2:end]
g2 = sum(delta.*b)/sum(b)

g = -g1 - g2 + g3


In [ ]:
saveres = results_df

In [ ]:
CSV.write("figures/results_uncert.csv", results_df)